### Setup Jupyter notebook
Setup Jupyter notebook to use (Python) modules in this project.

In [1]:
from pathlib import Path
import sys

# Make this path be the project's "base" directory, so we can include modules
notebook_directory_ancestor = Path.cwd().resolve().parent.parent
print(notebook_directory_ancestor)
core_code_directory = notebook_directory_ancestor / "CoreCode/"
is_core_code_directory_in_sys_path = str(core_code_directory) in sys.path
is_notebook_directory_ancestor_in_sys_path = str(notebook_directory_ancestor) in sys.path
print("Is CoreCode directory in sys.path?", is_core_code_directory_in_sys_path)
print("Is notebook directory's ancestor in sys.path?", is_notebook_directory_ancestor_in_sys_path)

/InServiceOfX
Is CoreCode directory in sys.path? False
Is notebook directory's ancestor in sys.path? False


In [2]:
if not is_core_code_directory_in_sys_path:
    sys.path.append(str(core_code_directory))
    
sys.path.append(str(notebook_directory_ancestor/ "Embeddings/"))

In [3]:
from CoreCode.MongoDBInterface.connect_to_client import connect_to_client
from CoreCode.MongoDBInterface.reconstruct_uri import reconstruct_minimal_uri
from CoreCode.Utilities.LoadEnvironmentFile import load_environment_file
load_environment_file()

In [4]:
import os, pymongo
example_uri_from_env = os.environ.get("EXAMPLE_MONGODB_URI")
parsed_uri_dict = pymongo.uri_parser.parse_uri(example_uri_from_env)

Enter your password for MongoDB.

In [5]:
parsed_uri_dict['password'] = ""
example_uri = reconstruct_minimal_uri(parsed_uri_dict)

In [6]:
client = connect_to_client(example_uri)
db = client.sample_mflix
collection = db.movies
print(type(client))
print(type(db))
print(type(collection))

<class 'pymongo.mongo_client.MongoClient'>
<class 'pymongo.database.Database'>
<class 'pymongo.collection.Collection'>


In [7]:
from Embeddings.Text.GenerateEmbedding import (QueryHuggingFaceFeatureExtraction)

In [8]:
token = os.environ.get("HUGGINGFACE_TOKEN")

In [9]:
query_feature_extraction = QueryHuggingFaceFeatureExtraction(token)

In [10]:
%%time
saved_some_docs = []
counter = 0
last_doc = None
for doc in collection.find({'plot':{"$exists": True}}).limit(50):
    try:
        doc['plot_embedding_hf'] = query_feature_extraction.query_feature_extraction(doc['plot'])
    except ValueError as err:
        print(err)
        print("counter: ", counter)
        last_doc = doc
    collection.replace_one({'_id': doc['_id']}, doc)
    saved_some_docs.append(doc)
    counter += 1

CPU times: user 5.04 s, sys: 148 ms, total: 5.19 s
Wall time: 4min 34s
